In [1]:
#Imprort des librairies nécessaires
import pandas as pd
import numpy as np

In [2]:
# Fichier 2019-parc-vehicules-routiers-2pr2.xls :
# Parc par département, région, source d'énergie (2011 à 2019)
# Comprends deux groupes de sheets au format légèrement distincts : avant et apres la fusion des regions en 2016

#Nom des colonnes
col_names_temp_2011=["Regions","Departements", "Ess-SuperE<6", "Ess-SuperE6-7", "Ess-SuperE>7", "Elec-Ess<6","Elec-Ess>6", "Gazole<6", "Gazole>6", "Ess-GPL>6", "Ess-GPL<6",
"Electricite","Autres+Non_detecte","Total"]
col_names_temp=["Regions","Departements", "Ess-SuperE<6", "Ess-SuperE6-7", "Ess-SuperE>7", "Elec-Ess<6","Elec-Ess>6", "Gazole<6", "Gazole>6", "Ess-GPL>6", "Ess-GPL<6",
"Electricite","Gazole-électricité","Autres+Non_detecte","Total"]

col_names= ["Année","Regions","Departements", "Essence+Superéthanol","Electricite-Essence","Gazole", "Essence-GPL",
"Electricite","Gazole-électricité","Autres+Non_detecte","Total"]


#Dico des regions et des departements :
regions = {"Auvergne-Rhône-Alpes":["Ain","Allier","Ardèche","Cantal","Drôme","Haute-Loire","Haute-Savoie","Isère","Loire","Puy-de-Dôme","Rhône","Savoie"],
           "Bourgogne-Franche-Comté":["Côte-d'Or","Doubs","Haute-Saône","Jura","Nièvre","Saône-et-Loire","Territoire de Belfort","Yonne"],
           "Bretagne":["Côtes-d'Armor","Finistère","Ille-et-Vilaine","Morbihan"],
           "Centre-Val de Loire":["Cher","Eure-et-Loir","Indre","Indre-et-Loire","Loiret","Loir-et-Cher"],
           "Corse":["Corse-du-Sud","Haute-Corse"],
           "Grand Est":["Ardennes","Aube","Bas-Rhin","Haute-Marne","Haut-Rhin","Marne","Meurthe-et-Moselle","Meuse","Moselle","Vosges"],
           "Hauts-de-France":["Aisne","Nord","Oise","Pas-de-Calais","Somme"],
           "Île-de-France":["Essonne","Hauts-de-Seine","Paris","Seine-et-Marne","Seine-Saint-Denis","Val-de-Marne","Val-d'Oise","Yvelines"],
           "Normandie":["Calvados","Eure","Manche","Orne","Seine-Maritime"], 
           "Nouvelle-Aquitaine":["Charente","Charente-Maritime","Corrèze","Creuse","Deux-Sèvres","Dordogne","Gironde","Haute-Vienne","Landes","Lot-et-Garonne","Pyrénées-Atlantiques","Vienne"],
           "Occitanie":["Ariège","Aude","Aveyron","Gard","Gers","Haute-Garonne","Hautes-Pyrénées","Hérault","Lot","Lozère","Pyrénées-Orientales","Tarn","Tarn-et-Garonne"],
           "Pays de la Loire":["Loire-Atlantique","Maine-et-Loire","Mayenne","Sarthe","Vendée"],
           "Provence-Alpes-Côte d'Azur":["Alpes-de-Haute-Provence","Alpes-Maritimes","Bouches-du-Rhône","Hautes-Alpes","Var","Vaucluse"]}
           


In [3]:
#Recuperation de chaque sheet dans un dico :

sheets=["1-1-2011","1-1-2012","1-1-2013","1-1-2014","1-1-2015","1-1-2016", "1-1-2017","1-1-2018","1-1-2019"]


excel_voit_part=dict()

for year in sheets:
    if year == '1-1-2011':
         excel_voit_part[year] = pd.read_excel("https://www.statistiques.developpement-durable.gouv.fr/sites/default/files/2019-04/2019-parc-vehicules-routiers-2pr2.xls", header = None, names=col_names_temp_2011, sheet_name=str(year))
    else:
        excel_voit_part[year] = pd.read_excel("https://www.statistiques.developpement-durable.gouv.fr/sites/default/files/2019-04/2019-parc-vehicules-routiers-2pr2.xls", header = None, names=col_names_temp, sheet_name=str(year))


In [4]:
for year in excel_voit_part :
    print(year, excel_voit_part[year].shape)

1-1-2011 (146, 14)
1-1-2012 (146, 15)
1-1-2013 (146, 15)
1-1-2014 (146, 15)
1-1-2015 (146, 15)
1-1-2016 (146, 15)
1-1-2017 (128, 15)
1-1-2018 (129, 15)
1-1-2019 (129, 15)


In [5]:
def get_reg(row):
    for reg, dep_list in regions.items():
         if str(row['Departements']) in dep_list:
             return reg
        
excel_voit_part_transfo = dict()

for year in excel_voit_part :
    sh = excel_voit_part[year]
    print(year, sh.shape)
    
    if year =="1-1-2018" or year=="1-1-2019" :
        #Supp des 4 premières lignes
        sh=sh[4:]
        print("Sup 4 prem", sh.shape)
        #Supp des lignes Total :
        sh = sh[sh.Departements != "Total"]
        print("Sup tot", sh.shape)
    else:
        #Supp des 3 premières lignes
        sh=sh[3:]
        print("Sup 3 prem", sh.shape)
        #Supp des lignes Total :
        sh = sh[sh['Departements'] != "Total :"]
        print("Sup tot", sh.shape)
 
    #Supp 4 dernières lignes
    sh=sh[:-4]
    print("Sup 4 der", sh.shape)

    #Suppression des lignes vides (lignes de séparation entre les regions)
    sh=sh.dropna(axis=0,how='all')
    print("Sup vides", sh.shape)

    #Recurération des regions 
    sh['Regions']=sh.apply(lambda row: get_reg(row), axis=1)
    
    #Rajout d'une columne "Gazole-électricité" dans le cas de l'année 2011
    if year =="1-1-2011" :
        sh["Gazole-électricité"] = np.nan
        
    #Remplacement des valeurs nulles par 0 (il ny en a que dans la colonnes galzole-elec)
    sh["Gazole-électricité"]=sh["Gazole-électricité"].fillna(0).astype(int)
    

    #Regroupement des colonnes Essence+Superéthanol,Electricite-Essence, Gazole, Essence-GPL :
    sh["Essence+Superéthanol"]= sh["Ess-SuperE<6"]+sh["Ess-SuperE6-7"]+sh["Ess-SuperE>7"]
    sh["Electricite-Essence"]= sh["Elec-Ess<6"]+sh["Elec-Ess>6"]
    sh["Gazole"]=sh["Gazole<6"]+sh["Gazole>6"]
    sh["Essence-GPL"]=sh["Ess-GPL>6"]+sh["Ess-GPL<6"]

    ### Test pour vérifier que les calculs son corrects
    sh["Total_test"] = sh["Essence+Superéthanol"]+sh["Electricite-Essence"]+sh["Gazole"]+sh["Essence-GPL"]+sh["Electricite"]+sh["Gazole-électricité"]+sh["Autres+Non_detecte"]
    print("Calculs bons", sh['Total'].isin(sh['Total_test']).value_counts())
        
    #Rajout d'une colonne année :
    sh["Année"]=str(year[4:])
    
    #Suppression des colonnes temp et re-arrangement dans le bon ordre
    sh=sh[col_names]
    print("Dropcol", sh.shape, "\n")

    
    #Sorting alphabetically
    sh=sh.sort_values('Departements')
    sh=sh.sort_values('Regions')
    
    #Reset de l'index
    sh=sh.reset_index(drop=True)
    
    excel_voit_part_transfo[year]=sh
    
    #Concatenation des années
    concat_excel_voit_part_transfo=pd.concat(excel_voit_part_transfo)
    #Renommage des colonnes parce que j'ai fait n'importe auoi au debut
    concat_excel_voit_part_transfo = concat_excel_voit_part_transfo.rename(columns={"Regions":"Régions","Departements" : "Départements" ,"Electricite-Essence":"Electricité-Essence","Electricite":"Electricité","Autres+Non_detecte":"Autres et non détectés"})
    #Ecriture du fichier
    concat_excel_voit_part_transfo.to_csv("voiture_par_dep.csv",sep=';', index = False)

1-1-2011 (146, 14)
Sup 3 prem (143, 14)
Sup tot (121, 14)
Sup 4 der (117, 14)
Sup vides (96, 14)
Calculs bons True    96
Name: Total, dtype: int64
Dropcol (96, 11) 

1-1-2012 (146, 15)
Sup 3 prem (143, 15)
Sup tot (121, 15)
Sup 4 der (117, 15)
Sup vides (96, 15)
Calculs bons True    96
Name: Total, dtype: int64
Dropcol (96, 11) 

1-1-2013 (146, 15)
Sup 3 prem (143, 15)
Sup tot (121, 15)
Sup 4 der (117, 15)
Sup vides (96, 15)
Calculs bons True    96
Name: Total, dtype: int64
Dropcol (96, 11) 

1-1-2014 (146, 15)
Sup 3 prem (143, 15)
Sup tot (121, 15)
Sup 4 der (117, 15)
Sup vides (96, 15)
Calculs bons True    96
Name: Total, dtype: int64
Dropcol (96, 11) 

1-1-2015 (146, 15)
Sup 3 prem (143, 15)
Sup tot (121, 15)
Sup 4 der (117, 15)
Sup vides (96, 15)
Calculs bons True    96
Name: Total, dtype: int64
Dropcol (96, 11) 

1-1-2016 (146, 15)
Sup 3 prem (143, 15)
Sup tot (121, 15)
Sup 4 der (117, 15)
Sup vides (96, 15)
Calculs bons True    96
Name: Total, dtype: int64
Dropcol (96, 11) 

1-1-

In [6]:
excel_voit_part_transfo['1-1-2011']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2011,Auvergne-Rhône-Alpes,Ain,121946,410,204179,1651,12,0,138,328336
1,2011,Auvergne-Rhône-Alpes,Allier,63665,114,113563,1061,8,0,22,178433
2,2011,Auvergne-Rhône-Alpes,Haute-Loire,35047,91,84359,930,2,0,27,120456
3,2011,Auvergne-Rhône-Alpes,Savoie,83682,319,149496,1760,8,0,61,235326
4,2011,Auvergne-Rhône-Alpes,Rhône,349385,1167,518104,5348,82,0,173,874259
...,...,...,...,...,...,...,...,...,...,...,...
91,2011,Île-de-France,Seine-Saint-Denis,215094,873,308838,3541,94,0,137,528577
92,2011,Île-de-France,Paris,365072,2748,325872,4077,247,0,245,698261
93,2011,Île-de-France,Hauts-de-Seine,341680,2531,414463,3125,228,0,197,762224
94,2011,Île-de-France,Val-de-Marne,266584,1092,275643,3326,63,0,97,546805


In [7]:
excel_voit_part_transfo['1-1-2012']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2012,Auvergne-Rhône-Alpes,Ain,115414,548,214281,1634,22,0,137,332036
1,2012,Auvergne-Rhône-Alpes,Allier,58597,155,116457,1035,15,0,18,176277
2,2012,Auvergne-Rhône-Alpes,Haute-Loire,32480,104,86254,940,5,0,25,119808
3,2012,Auvergne-Rhône-Alpes,Savoie,78996,391,155943,1726,11,0,65,237132
4,2012,Auvergne-Rhône-Alpes,Rhône,327699,1479,537061,5465,125,0,174,872003
...,...,...,...,...,...,...,...,...,...,...,...
91,2012,Île-de-France,Seine-Saint-Denis,201954,1136,324129,3540,93,0,133,530985
92,2012,Île-de-France,Paris,338109,3436,336198,3881,257,1,229,682111
93,2012,Île-de-France,Hauts-de-Seine,324636,3188,432079,3038,921,2,182,764046
94,2012,Île-de-France,Val-de-Marne,250200,1464,286963,3240,62,0,110,542039


In [8]:
excel_voit_part_transfo['1-1-2013']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2013,Auvergne-Rhône-Alpes,Ain,112287,725,224203,1545,40,101,134,339035
1,2013,Auvergne-Rhône-Alpes,Allier,55694,228,119449,976,26,53,21,176447
2,2013,Auvergne-Rhône-Alpes,Haute-Loire,31272,127,88518,898,15,48,25,120903
3,2013,Auvergne-Rhône-Alpes,Savoie,76546,486,161699,1625,27,86,63,240532
4,2013,Auvergne-Rhône-Alpes,Rhône,315335,1903,556762,5224,193,340,175,879932
...,...,...,...,...,...,...,...,...,...,...,...
91,2013,Île-de-France,Seine-Saint-Denis,193955,1536,342315,3274,100,130,133,541443
92,2013,Île-de-France,Paris,319425,4137,346368,3404,397,640,203,674574
93,2013,Île-de-France,Hauts-de-Seine,312379,4032,448512,2808,2607,490,170,770998
94,2013,Île-de-France,Val-de-Marne,240331,2061,299182,3021,101,147,107,544950


In [9]:
excel_voit_part_transfo['1-1-2014']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2014,Auvergne-Rhône-Alpes,Ain,108777,995,231986,1419,100,233,139,343649
1,2014,Auvergne-Rhône-Alpes,Allier,52279,316,121846,902,43,130,18,175534
2,2014,Auvergne-Rhône-Alpes,Haute-Loire,29941,185,90111,838,28,105,23,121231
3,2014,Auvergne-Rhône-Alpes,Savoie,73994,706,165266,1434,71,205,58,241734
4,2014,Auvergne-Rhône-Alpes,Rhône,301930,2735,570742,4734,405,932,173,881651
...,...,...,...,...,...,...,...,...,...,...,...
91,2014,Île-de-France,Seine-Saint-Denis,182189,2328,356093,2779,169,333,128,544019
92,2014,Île-de-France,Paris,300263,5487,355598,2611,667,1467,157,666250
93,2014,Île-de-France,Hauts-de-Seine,298803,6102,463743,2446,3744,1154,158,776150
94,2014,Île-de-France,Val-de-Marne,229108,3092,306646,2645,182,377,101,542151


In [10]:
excel_voit_part_transfo['1-1-2015']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2015,Auvergne-Rhône-Alpes,Ain,104949,1292,236497,1318,167,356,138,344717
1,2015,Auvergne-Rhône-Alpes,Allier,49179,398,122322,845,80,171,17,173012
2,2015,Auvergne-Rhône-Alpes,Haute-Loire,28487,240,90456,797,41,143,17,120181
3,2015,Auvergne-Rhône-Alpes,Savoie,71245,918,166492,1326,126,283,57,240447
4,2015,Auvergne-Rhône-Alpes,Rhône,286668,3655,576158,4382,693,1333,172,873061
...,...,...,...,...,...,...,...,...,...,...,...
91,2015,Île-de-France,Seine-Saint-Denis,169105,3263,364539,2450,305,596,115,540373
92,2015,Île-de-France,Paris,277504,7046,351006,2198,1117,1801,130,640802
93,2015,Île-de-France,Hauts-de-Seine,281877,8803,467666,2197,5105,1849,137,767634
94,2015,Île-de-France,Val-de-Marne,216054,4173,310097,2394,299,586,92,533695


In [11]:
excel_voit_part_transfo['1-1-2016']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2016,Auvergne-Rhône-Alpes,Ain,103831,1671,238399,1285,335,567,135,346223
1,2016,Auvergne-Rhône-Alpes,Allier,47568,477,121451,795,147,263,15,170716
2,2016,Auvergne-Rhône-Alpes,Haute-Loire,27982,280,89773,767,87,179,18,119086
3,2016,Auvergne-Rhône-Alpes,Savoie,70439,1131,166576,1269,244,405,49,240113
4,2016,Auvergne-Rhône-Alpes,Rhône,278634,4806,573420,4212,1123,1896,176,864267
...,...,...,...,...,...,...,...,...,...,...,...
91,2016,Île-de-France,Seine-Saint-Denis,159978,4460,378995,2336,490,1001,108,547368
92,2016,Île-de-France,Paris,257856,9319,344236,2016,1425,2639,120,617611
93,2016,Île-de-France,Hauts-de-Seine,271857,11887,467357,2088,6709,3096,123,763117
94,2016,Île-de-France,Val-de-Marne,206412,5599,310535,2300,530,974,84,526434


In [12]:
excel_voit_part_transfo['1-1-2017']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2017,Auvergne-Rhône-Alpes,Ain,104614,2179,237397,1237,567,640,121,346755
1,2017,Auvergne-Rhône-Alpes,Allier,47848,652,119337,756,225,326,16,169160
2,2017,Auvergne-Rhône-Alpes,Haute-Loire,28374,384,87961,735,148,211,16,117829
3,2017,Auvergne-Rhône-Alpes,Savoie,71702,1437,163966,1199,412,449,51,239216
4,2017,Auvergne-Rhône-Alpes,Rhône,277279,6213,565877,4074,1771,2044,164,857422
...,...,...,...,...,...,...,...,...,...,...,...
91,2017,Île-de-France,Seine-Saint-Denis,153845,5819,374772,2269,808,1206,100,538819
92,2017,Île-de-France,Paris,250655,11756,337116,1830,1984,2646,124,606111
93,2017,Île-de-France,Hauts-de-Seine,265887,15314,464394,1876,7681,3137,123,758412
94,2017,Île-de-France,Val-de-Marne,197535,7538,308054,2203,816,1132,83,517361


In [13]:
excel_voit_part_transfo['1-1-2018']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2018,Auvergne-Rhône-Alpes,Ain,109049,2964,233993,1169,800,654,120,348749
1,2018,Auvergne-Rhône-Alpes,Allier,49018,895,116487,742,304,330,20,167796
2,2018,Auvergne-Rhône-Alpes,Haute-Loire,29971,549,85832,689,205,218,26,117490
3,2018,Auvergne-Rhône-Alpes,Savoie,75532,2052,160449,1141,611,462,48,240295
4,2018,Auvergne-Rhône-Alpes,Rhône,286145,8471,557945,3991,2630,2015,182,861379
...,...,...,...,...,...,...,...,...,...,...,...
91,2018,Île-de-France,Seine-Saint-Denis,152492,7767,376738,2289,1346,1317,95,542044
92,2018,Île-de-France,Paris,243994,15595,317999,1720,3064,2314,132,584818
93,2018,Île-de-France,Hauts-de-Seine,268421,19019,462424,1804,8723,2870,154,763415
94,2018,Île-de-France,Val-de-Marne,196873,9988,299139,2150,1241,1095,78,510564


In [14]:
excel_voit_part_transfo['1-1-2019']

,Année,Regions,Departements,Essence+Superéthanol,Electricite-Essence,Gazole,Essence-GPL,Electricite,Gazole-électricité,Autres+Non_detecte,Total
0,2019,Auvergne-Rhône-Alpes,Ain,116161,4113,228091,1166,1095,683,116,351425
1,2019,Auvergne-Rhône-Alpes,Allier,52047,1314,112329,757,407,346,24,167224
2,2019,Auvergne-Rhône-Alpes,Haute-Loire,32872,767,83346,668,264,244,27,118188
3,2019,Auvergne-Rhône-Alpes,Savoie,81617,3003,155946,1098,804,503,46,243017
4,2019,Auvergne-Rhône-Alpes,Rhône,301835,11242,541211,3936,3470,1985,184,863863
...,...,...,...,...,...,...,...,...,...,...,...
91,2019,Île-de-France,Seine-Saint-Denis,154608,9851,369902,2271,1754,1419,100,539905
92,2019,Île-de-France,Paris,245085,19568,300052,1670,4690,2067,138,573270
93,2019,Île-de-France,Hauts-de-Seine,279118,23093,441906,1735,10972,2448,161,759433
94,2019,Île-de-France,Val-de-Marne,200066,12620,285431,2075,1633,1105,75,503005


In [15]:
### Precedent test pour voir calculer le nb de valeur nulles dans chaques colonnes:
concat_excel_voit_part_transfo.isnull().sum()

Année                     0
Régions                   0
Départements              0
Essence+Superéthanol      0
Electricité-Essence       0
Gazole                    0
Essence-GPL               0
Electricité               0
Gazole-électricité        0
Autres et non détectés    0
Total                     0
dtype: int64